In [1]:
import pandas as pd

def load_large_excel(file_path):
    """
    Загружает большой Excel-файл с оптимизацией памяти.
    
    Параметры:
        file_path (str): Путь к файлу Excel (.xlsx или .xls).
    
    Возвращает:
        pd.DataFrame или None: Загруженные данные или None в случае ошибки.
    """
    try:
        # Определяем типы столбцов заранее, чтобы уменьшить использование памяти
        dtype_spec = {
            'Reposts': 'int32',    # Экономит память по сравнению с int64
            'Views': 'int32',
            'Link': 'string',       # Оптимизированная строка (вместо object)
            'Text': 'string'
        }
        
        # Чтение файла с оптимизацией
        df = pd.read_excel(
            file_path,
            engine='openpyxl',      # Лучше для .xlsx (для .xls можно 'xlrd')
            dtype=dtype_spec,       # Задаём типы сразу при чтении
            usecols=['Reposts', 'Views', 'Link', 'Text'],  # Читаем только нужные столбцы
            # na_values=['NA', 'N/A', ''],  # Обработка пропущенных значений
            keep_default_na=False   # Уменьшает нагрузку на память
        )
        
        # Заполняем возможные пропуски (если не критичны)
        df['Reposts'] = df['Reposts'].fillna(0).astype('int32')
        df['Views'] = df['Views'].fillna(0).astype('int32')
        
        return df
    
    except FileNotFoundError:
        print(f"Ошибка: Файл не найден — {file_path}")
    except Exception as e:
        print(f"Ошибка при загрузке Excel: {e}")
    
    return None

In [3]:
file_path = "dset_vk_2024.xlsx"  # Укажите свой путь
data = load_large_excel(file_path)

if data is not None:
    print("Данные успешно загружены:")
    print(f"Всего строк: {len(data)}")
    print(data.head())

Данные успешно загружены:
Всего строк: 156672
   Reposts  Views                               Link  \
0        0    244  https://vk.com/wall-6121824_29834   
1        0    197  https://vk.com/wall-6121824_29833   
2        1    317  https://vk.com/wall-6121824_29832   
3        0    260  https://vk.com/wall-6121824_29831   
4        0    354  https://vk.com/wall-6121824_29830   

                                                Text  
0  Поздравление врио Главы Республики Коми Ростис...  
1  На Украине признали самый низкий уровень рожда...  
2  Жителей России призвали воздержаться от исполь...  
3  🇷🇺 Жители Коми выбирают название новой кадрово...  
4  🏆 Для титулованных спортсменов из Республики К...  


In [118]:
import spacy
import re

# Загружаем модель для русского языка
nlp = spacy.load("ru_core_news_sm")

# def merge_dicts(lhs, rhs):
#     for k in ["action", "subjects"]:
#         lhs[k] += rhs[k]
#     return lhs

def token_chunk(token):
    return [t.text for t in token.subtree]

def find_mods(token):
    modifiers = set()
    for token_ in token.children:
        if token_.dep_ in ["nmod", "amod", "advmod"]:
            modifiers.add(token_.text)
            modifiers.update(find_mods(token_))
    return modifiers

def explote_subtree(token):
    modifiers = []
    elements = []
    for token_ in token.children:
        if token_.dep_ in ["nmod", "amod", "advmod"]: 
            modifiers.append([token_.text, find_mods(token_)])
        if token_.dep_ == "conj":
            elements += explote_subtree(token_)
    elements.append({"item" : token.text, "modifiers" : modifiers})
    return elements

def find_iobj(token):
    iobjs = []
    for token_ in token.subtree:
        if token != token_:
            if token_.dep_ in ["iobj", "obj", "nsubj:pass"]:
                iobjs.append(explote_subtree(token_))
    if iobjs == []:
        return None
    else:
        return iobjs
    
def find_subj(token):
    subjs = []
    for token_ in token.subtree:
        if token != token_:
            if token_.dep_ in ["nsubj"]:
                subjs.append(explote_subtree(token_))
    if subjs == []:
        return None
    else:
        return subjs
    
def search_from_root(token):
    this_predicate = {"action" : [token.text], "subjects" : [], "objects" : []}
    # непрямые объекты (если через что-то обращены), обрабатываются отдельно
    iobjs = find_iobj(token)
    if iobjs is not None:
        this_predicate["objects"] += iobjs
    subjs = find_subj(token)
    if subjs is not None:
        this_predicate["subjects"] += subjs

    for token_ in token.children:
        if token_.dep_ in ["xcomp", "csubj:pass", "conj"]: # то надо воспринимать его, как еще один рут
            this_predicate["action"].append(token_.text)
    return this_predicate

def analyze_sentence(sentence):
    print(sentence)
    """Анализирует предложение и возвращает подлежащее и сказуемое."""
    doc = nlp(sentence)
    
    predicates = []  # сказуемое (глагол или связка)
    ne = []
    
    for token in doc:
        if token.dep_ in ["ROOT"]:
            predicates.append(search_from_root(token))

        print(token, token.pos_, token.morph.get("Case", []), "\t", token.dep_, "  |  ", token.head)

        if token.pos_ == "PROPN":
            nes = explote_subtree(token)
            ne_reprs = []
            for item in nes:
                mod_reprs = []
                for mod in item["modifiers"]:
                    main_mod = mod[0]
                    mod_props = ' '.join(elm for elm in mod[1])
                    mod_repr = main_mod + " " + mod_props
                    mod_reprs.append(mod_repr)
                ne_reprs.append(item["item"] + " " + " ".join(mod_reprs))
            ne.append(' '.join(ne_reprs).strip())          

    
    return {"predicates": predicates, "named entities" : ne}

def get_text_main_part(text):
    for sep in ['. ', '\n']:
        text =  text.replace(sep, '¬')
    sentences = [sent for sent in text.split('¬')]  # Разбиваем текст на предложения

    anres = []
    for sent in sentences:
        sent = re.sub(r'[^а-яё.,:()\[\]0-9А-Я]', ' ', sent)
        sent = re.sub('\s+', ' ', sent)
        analysis = analyze_sentence(sent)
        anres.append(analysis)
        print(analysis)
    return anres

<>:109: SyntaxWarning: invalid escape sequence '\s'
<>:109: SyntaxWarning: invalid escape sequence '\s'
C:\Users\User\AppData\Local\Temp\ipykernel_1564\1157283608.py:109: SyntaxWarning: invalid escape sequence '\s'
  sent = re.sub('\s+', ' ', sent)


In [119]:
ans = []
for text in data['Text'][0:20]:
    ans.append(get_text_main_part(text))

Поздравление врио Главы Республики Коми Ростислава Гольдштейна с наступающим Новым годом
Поздравление NOUN ['Nom'] 	 ROOT   |   Поздравление
врио NOUN ['Gen'] 	 nmod   |   Поздравление
Главы NOUN ['Gen'] 	 nmod   |   врио
Республики PROPN ['Gen'] 	 nmod   |   Главы
Коми PROPN ['Gen'] 	 appos   |   врио
Ростислава PROPN ['Gen'] 	 flat:name   |   Коми
Гольдштейна PROPN ['Gen'] 	 flat:name   |   Коми
с ADP [] 	 case   |   годом
наступающим VERB ['Ins'] 	 amod   |   годом
Новым ADJ ['Ins'] 	 amod   |   годом
годом NOUN ['Ins'] 	 nmod   |   Поздравление
{'predicates': [{'action': ['Поздравление'], 'subjects': [], 'objects': []}], 'named entities': ['Республики', 'Коми', 'Ростислава', 'Гольдштейна']}
На Украине признали самый низкий уровень рождаемости на планете 000 
На ADP [] 	 case   |   Украине
Украине PROPN ['Loc'] 	 obl   |   признали
признали VERB [] 	 ROOT   |   признали
самый ADJ ['Acc'] 	 amod   |   низкий
низкий ADJ ['Acc'] 	 amod   |   уровень
уровень NOUN ['Acc'] 	 obj   |   при